## Notebook to Run QNBM on IBM Simulator

### IMPORTS

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ, execute, transpile
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.visualization import *
from qiskit.visualization import plot_histogram
from qiskit.providers.ibmq.runtime import UserMessenger
from testqneuronQS import qneuron_main
from cardinality import subset_distribution
from utils import dec2bin, comb 
import scipy as sp
import pandas as pd
from numpy import pi
from importlib import reload
import math

In [2]:
import testqneuron

### DEFINE GENERAL ARGS

In [3]:
problem_seed = 123 #Random initial param seed 
training_seed = 123 #Random training seed 
eps = 1e-1 #Training step epsilon  
lr = 0.2 #Training learning rate 
epsilon = 1e-16 #KL Divergence metric epsilon 
maxiter = 500 #Number of iterations 
simulator = Aer.get_backend('aer_simulator_statevector') #IBM Backend 
user_messenger = UserMessenger() #Backend 

### DEFINE QNBM (IN, HID, OUT) 

In [4]:
k = 1   #Activation sigmoid constant 
n_input_nodes = 4 #Number of input neurons 
hidden1 = 0 #Number of hidden neurons 
n_output_nodes = 5 #Number of output neurons 

### RUN QNBM 

In [5]:
#Define Random Initial Parameters 
np.random.seed(problem_seed)
if hidden1 > 0: 
    initial_weights = np.random.rand((n_input_nodes * hidden1) + (hidden1 * n_output_nodes))
    structure = [n_input_nodes, hidden1, n_output_nodes]
    
else: 
    initial_weights = np.random.rand(n_input_nodes * n_output_nodes)
    structure = [n_input_nodes, n_output_nodes]
initial_biases = np.random.rand(hidden1 + n_output_nodes)


#Define Cardinality Target Distribution 
cardinality = 1
training_distribution_dict = subset_distribution(n_output_nodes, cardinality)

{'00001': 0.2, '00010': 0.2, '00100': 0.2, '01000': 0.2, '10000': 0.2}


In [ ]:
#Define param gradient calculation functions 

def paramsplus(params, index):
    new_params = params
    new_params[index] = new_params[index] + np.pi/2;
    return new_params

def paramsminus(params, index):
    new_params = params
    new_params[index] = new_params[index] - np.pi/2;
    return new_params

In [9]:
#Define model and training distribution 
qnn = QuantumNeuralNetwork(structure, k)
training_distribution = qnn.ProbsDict_to_ProbsArray(training_distribution_dict, 1e-16)

[1.e-16 2.e-01 2.e-01 1.e-16 2.e-01 1.e-16 1.e-16 1.e-16 2.e-01 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 2.e-01 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16]


In [11]:
#Run main script and return optimal params and kl divergence 
params, kl = testqneuron.main(simulator, user_messenger, training_distribution, structure, k, initial_weights, initial_biases, maxiter, eps, lr, training_seed)

params:  [0.69646919 0.28613933 0.22685145 0.55131477 0.71946897 0.42310646
 0.9807642  0.68482974 0.4809319  0.39211752 0.34317802 0.72904971
 0.43857224 0.0596779  0.39804426 0.73799541 0.18249173 0.17545176
 0.53155137 0.53182759 0.63440096 0.84943179 0.72445532 0.61102351
 0.72244338]
shots used:  102400
Normalized weights and biases:  [0.10940112 0.04494666 0.03563374 0.08660032 0.11301392 0.06646141
 0.15405808 0.1075728  0.07554461 0.06159368 0.05390628 0.11451886
 0.06889077 0.00937418 0.06252465 0.11592405 0.02866573 0.0275599
 0.08349589 0.08353928] [0.88504963 0.9188266  0.89919534 0.88137751 0.8988793 ]
counts:  {'0111100101': 82, '1000001100': 4, '1111110000': 2426, '1101110101': 244, '0100011011': 12, '0001000110': 3, '0100000001': 3, '1110010100': 7, '0001111100': 237, '1101100001': 204, '0110010010': 69, '1010100001': 30, '0011110100': 93, '1000101000': 19, '1111101000': 2414, '1100100000': 26, '1000101110': 134, '1111101110': 129, '1011100000': 806, '1011111000': 848, 

In [ ]:
local_params = np.stack(local_params)
index = local_kl.index(np.min(local_kl))
params = local_params.copy()[index]

In [ ]:
n_params = len(params)
weights = params.copy()[0:len(initial_weights)]
biases = params.copy()[len(initial_weights):]
verifycirc = qnn.CreateCircuit(weights,biases)
transpiled_circuit = transpile(verifycirc, local_sim)
verifyjob = local_sim.run(transpiled_circuit, shots = 10000)
result = verifyjob.result().get_counts()
probs = qnn.FindProbs(result)